Tis is a simulation notebook to add few commands to the input file which was produced by CHARMM-GUI system

It is advisable to perform all the equilibration steps that CHARMM-GUI makes plus one short prduction run, before seting up the QM/MM simulation input file.

One can do standard dynamics run, but few additions need to be made:

1. add the environment to define the location of GAMESS files

!we need addlink commands if we break the covalent bond by splitting QM and MM regions

addl qqh1 mpep 1 c mpep 1 ca

! se need to select a QM region - this is currently by a normal CHARMM selection command
! in the future there will be a QM selector notebook
define qm sele atom mpep 1 c  .or. atom mpep 1 ot1 - 
          .or. atom mpep 1 ot2 end

!Necessary environment variables for a standard DFT calculations.
!The program is complaining if it needs more so one can add it here!

! names in quotes so they are lowercase
envi input   "simulation.str"
envi ericfmt "ericfmt.dat"
envi output  "simulation.gms"
envi punch   "test.pun"
envi dictnry "test.f10"
envi work15  "test.f15"
envi dasort  "test.f20"
envi dftgrid "test.f21"
envi dftints "test.f22"

ericfmt.dat is a in a GAMESS distribution, but also availble here on the test folder
simulation.str file looks like this:

 $CONTRL COORD=UNIQUE NOSYM=1 ICHARG=-1
         SCFTYP=RHF
         dfttyp=b3lyp
 !        runtyp=prop
 !        nprint=-5
          RUNTYP=GRADIENT
 !        RUNTYP=OPTIMIZE
 !        EXETYP=CHECK
 !        MOLPLT=.TRUE.
 $END
 $SYSTEM MEMORY=1500000 TIMLIM=100000 $END
 $BASIS  
 !         GBASIS=AM1
 !         GBASIS=STO NGAUSS=3
          GBASIS=N31 NGAUSS=6 
 !         NDFUNC=3 NPFUNC=3
 !         DIFFSP=.TRUE. DIFFS=.TRUE.
 $END
 $SCF     DIRSCF=.True. diis=.true. $END
 $STATPT  NSTEP=100 OPTTOL=0.00000001 $END
 $dft sg1=.t. $end
 $DATA


 $END
 !$ELPOT IEPOT=1 where=pdc $END

2. after the link atoms are added (if needed) and GAMESS environment is defined and QM region is selected one runs the initial GAMESS command, so everything after this command is using QM/MM forces and energies

3. the simulation additions:
Usually these are lenghty calculations so we want to save every few steps into a trajectory file

write psf card name mini.psf
open write file unit 1 name mini.dcd
the following additions for trajectoy is the same for both the minimization comand or dynamics command

[mini abnr|dynamics] iuncrd 1 nsavc 1

NOTE: only *mini abnr* supports uncrd,nsavc keywords!

after this we have a psf and trajectory file for further QM/MM calcualtions.
Both can be used to view after or during the simulation is running or finished

In [ ]:
%%script bash
# define executable:
charmm_exe="/usr/local/apptainer/apptainer_deb/charmm_debs9.sif"
# one wants to run this in parallel with the following command
# NOTE: the container is build wth the OpenMPI library so it needs this library on the host system
# this is usually achieved by loading appropriate module (eg. module load OpenMPI)
mpirun -n 32 singularity run -B `pwd` $charmm_exe -i simulation.inp > simulation.out
# usually the above command goes into a batch system script!
# on some systems -B 1pwd` is not needed ??

In [1]:
# if only QM atoms are needed to show one can select them in the vmd menu
# select graphics and then representations then specify names in the Selected Atoms
# window. Use capital letters for atom names, eg name C CA ...
!vmd mini.psf mini.dcd
# there is another way to extract the QM region from the QM/MM calculations using simulation.gms file
# but this probably need another notebook, coming soon...

Info) VMD for LINUX, version 1.9.4a57 (September 21, 2023)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 32 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 AVX512F AVX512CD HT 
Info) Free system memory: 49GB (79%)
Info) OpenGL renderer: AMD Radeon Graphics (raphael_mendocino, LLVM 16.0.6, DRM 3.54, 6.5.7-gentoo)
Info)   Features: STENCIL MSAA(4) MDE CVA MTX NPOT PP PS GLSL(OVFS) 
Info)   Full GLSL rendering mode is available.
Info)   Textures: 2-D (16384x16384), 3-D (1024x1024x1024), Multitexture (8)
Info) Internal command editing disabled, external rlwrap in use.
Info) Dynamically loaded 3